In [1]:
import numpy as np
import pandas as pd
import spotipy as sp
import numbers
import spotipy.util as util

SPOTIPY_REDIRECT_URI='http://localhost:8888/callback'
SPOTIPY_CLIENT_SECRET='c02f97ddd119470d8e24b290722c7490'
SPOTIPY_CLIENT_ID='647ffb8c163b4882be0cafe4bf1352d5'
USERNAME='lh7smhcywr18vctg4xmnoehzu'
SCOPE='user-library-read'

token = util.prompt_for_user_token(username=USERNAME,
                           scope=SCOPE,
                           client_id=SPOTIPY_CLIENT_ID,
                           client_secret=SPOTIPY_CLIENT_SECRET,
                           redirect_uri=SPOTIPY_REDIRECT_URI)

s = sp.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-library-read&client_id=647ffb8c163b4882be0cafe4bf1352d5&response_type=code in your browser


Enter the URL you were redirected to: http://localhost:8888/callback?code=AQDWXXe8yyirS5nmXDDsM9yn1hr0YToJVgYwFGYFivIo7FUYe5-otTscWR3zcBiA8lgFtuYe-gVen0JGZYpVE_MkTzBAc_9FrfowrFOFpvv2HtRlqJWiLKZR_b7kk2A58GQhm1-zRua90ulYOu0c1vQzXuVaprtvYzEak4ErVJpCiQgNv1u08iVB8ryk2nOxk4Yt0FK4Dm24cZVKm0IquWzFxLV_XfE




In [2]:
def get_track_ids(tracks, ide=True):
    """Collect all track id's in a list of tracks"""
    
    if ide:
        return [track['track']['id'] for track in tracks]
    else:
        return [track['id'] for track in tracks]


def get_audio_features(ids):
    """Collect all audio features in a list from a given list of track id's"""
    audio_feats = []

    for i in range(0, len(ids), 100):
        audio_feats += s.audio_features(ids[i:i + 100])

    return audio_feats

def getDataframe(features):
    matrix = []
    metamatrix = []
    for track in features:
        try:
            matrix += [[x for x in track.values() if isinstance(x, numbers.Number)]]
            metamatrix += [[x for x in track.values() if not isinstance(x, numbers.Number)]]
        except:
            pass
        
    return np.array(matrix), metamatrix



def get_random_sample(genre, iters = 20):
    """Main function"""
    query = s.search(q='genre:{0}'.format(genre))
    tracks = []
    
    i = 0
    
    while query['tracks']['next']:
        tracks.extend(query['tracks']['items'])
        query = s.next(query['tracks'])
        
        i += 1
        
        if i == iters:
            break
            
    feature_names = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
     'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
  
    # These steps are not computationally expensive
    track_ids = get_track_ids(tracks, ide=False)
    audio_feats = get_audio_features(track_ids)
    
    matrix_of_features, metamatrix = getDataframe(audio_feats)
    df = pd.DataFrame(matrix_of_features)
    df.columns = feature_names
    
    return df, metamatrix

In [3]:
# Laurens: vaporwave, classical, hiphop, house
# Pieter:  rock,      punk,      jazz,   funk
a1, b1 = get_random_sample("rock", 1000)
a2, b2 = get_random_sample("punk", 1000)
a3, b3 = get_random_sample("jazz", 1000)
a4, b4 = get_random_sample("funk", 1000)

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [5]:
a1.to_pickle('rock_features')
a2.to_pickle('punk_features')
a3.to_pickle('jazz_features')
a4.to_pickle('funk_features')

In [10]:
import pickle

pickle.dump(b1, open('rock_metafeatures', 'wb'))
pickle.dump(b2, open('punk_metafeatures', 'wb'))
pickle.dump(b3, open('jazz_metafeatures', 'wb'))
pickle.dump(b4, open('funk_metafeatures', 'wb'))